In [1]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date

In [3]:
consents_pids = SubjectConsent.objects.values_list('screening_identifier', flat=True).distinct()


#Consented HUU
consented_md = MaternalDataset.objects.filter(protocol='Tshilo Dikotla',
                                              mom_hivstatus='HIV-uninfected',
                                              screening_identifier__in=consents_pids)

consented_md_pids = consented_md.values_list('study_maternal_identifier', flat=True)

#Unconsented HUU

td_uncosent_md = MaternalDataset.objects.filter(protocol='Tshilo Dikotla',
                                      mom_hivstatus='HIV-uninfected').exclude(
    screening_identifier__in=consents_pids)

td_uncosent_md_pids = td_uncosent_md.values_list('study_maternal_identifier', flat=True)


In [4]:
#not_randomized
non_rando = WorkList.objects.filter(study_maternal_identifier__in=td_uncosent_md_pids, assigned=None)

# WorkList.objects.first().__dict__
print(non_rando.count())

67

In [5]:
#unconsented_HUU call logs
td_unconsent_log_entries = LogEntry.objects.filter(study_maternal_identifier__in=td_uncosent_md_pids)
td_unconsent_log_entries.count()


df = pd.DataFrame([td.__dict__ for td in td_unconsent_log_entries ])
# df.to_csv('/Users/imosweu/td_unconsented.csv', encoding='utf-8')
df.to_csv('/home/flourish/td_unconsented.csv', encoding='utf-8')

In [ ]:
unconsent_log_entry_ids = list(set(td_unconsent_log_entries.values_list('study_maternal_identifier', flat=True)))


#success contact unconsented huu
success_log_entries = LogEntry.objects.filter(~Q(phone_num_success=['none_of_the_above']),
                                             study_maternal_identifier__in=td_uncosent_md_pids)

success_log_entries_ids = success_log_entries.values_list('study_maternal_identifier', flat=True)

success_df = pd.DataFrame([td.__dict__ for td in success_log_entries ])
# df.to_csv('/Users/imosweu/td_unconsented.csv', encoding='utf-8')
success_df.to_csv('/home/flourish/td_unconsent_success_contact.csv', encoding='utf-8')
success_log_entries.count()


In [17]:
#Final decline
declined = LogEntry.objects.filter(study_maternal_identifier__in=success_log_entries_ids,
                                   appt='No')

declined_ids = declined.values_list('study_maternal_identifier', flat=True)


final_decline = []
for a in declined_ids:
    dec_logs = LogEntry.objects.filter(study_maternal_identifier=a)
    for d in dec_logs:
        try:
            LogEntry.objects.get(study_maternal_identifier=d.study_maternal_identifier,
                                 call_datetime__gt=d.call_datetime, appt='Yes')
        except LogEntry.DoesNotExist:
            final_decline.append(d)
            
declined_df = pd.DataFrame([td.__dict__ for td in final_decline ])
declined_df.to_csv('/home/flourish/td_declined_contact.csv', encoding='utf-8')

len(final_decline)

In [ ]:
may_not_call = LogEntry.objects.filter(study_maternal_identifier__in=success_log_entries_ids,
                                       may_call__icontains='no_')

may_not_call_ids = may_not_call.values_list('study_maternal_identifier', flat=True)

final_may_not_call = []
for m in may_not_call_ids:
    dec_logs = LogEntry.objects.filter(study_maternal_identifier=m)
    for d in dec_logs:
        try:
            LogEntry.objects.get(study_maternal_identifier=d.study_maternal_identifier,
                                 call_datetime__gt=d.call_datetime, may_call__icontains='no_')
        except LogEntry.DoesNotExist:
            final_may_not_call.append(d)
            
final_may_not_call_df = pd.DataFrame([td.__dict__ for td in final_may_not_call ])
final_may_not_call_df.to_csv('/home/flourish/td_final_may_not_call.csv', encoding='utf-8')

In [ ]:
#unsuccesssuccess contact unconsented huu
unsucessful_log_entries = LogEntry.objects.filter(phone_num_success=['none_of_the_above'],
                                                 study_maternal_identifier__in=td_uncosent_md_pids).exclude(
    study_maternal_identifier__in=success_log_entries_ids)

unsucessful_log_entries.count()

unsuccess_df = pd.DataFrame([td.__dict__ for td in unsucessful_log_entries ])
# df.to_csv('/Users/imosweu/td_unconsented.csv', encoding='utf-8')
unsuccess_df.to_csv('/home/flourish/td_unsuccess_contact.csv', encoding='utf-8')